# GPT-2 архитектурасын Llama 2 архитектурасына түрлендіру
# Преобразование архитектуры GPT - 2  в Llama 2

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/gpt2-to-llama2-llama3.webp?1" width=1000px>

In [3]:
from importlib.metadata import version

pkgs = [
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

torch version: 2.5.1


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

##   LayerNorm қабатын RMSNorm қабатына ауыстыру
##  Заменить слой LayerNorm на слой RMSNorm

* Алдымен біз LayerNorm қабатын Root Mean Square Layer Normalization (RMSNorm) қабатымен ауыстырамыз.
* LayerNorm кірістерді орташа мән мен дисперсия арқылы қалыпқа келтіреді, ал RMSNorm тек квадраттық орташа мәнді (root mean square) пайдаланады, бұл есептеу тиімділігін арттырады.
* RMSNorm операциясы:    
$x$ — кіріс,    
$\gamma$ — үйренілетін параметр (вектор),   
$\epsilon$ — нөлге бөлу қатесінен сақтану үшін қолданылатын кіші тұрақты шама:

$$y_i = \frac{x_i}{\text{RMS}(x)} \gamma_i, \quad \text{мұндағы} \quad \text{RMS}(x) = \sqrt{\epsilon + \frac{1}{n} \sum x_i^2}$$
---
* Сначала мы заменяем слой LayerNorm на Root Mean Square Layer Normalization (RMSNorm).
* LayerNorm нормализует входы, используя среднее и дисперсию, тогда как RMSNorm использует только квадратный корень из среднего квадрата (root mean square), что повышает вычислительную эффективность.
* Операция RMSNorm определяется следующим образом:   
$x$ — вход,   
$\gamma$ — обучаемый параметр (вектор),  
$\epsilon$ — малая константа, предотвращающая деление на ноль:

$$y_i = \frac{x_i}{\text{RMS}(x)} \gamma_i, \quad \text{где} \quad \text{RMS}(x) = \sqrt{\epsilon + \frac{1}{n} \sum x_i^2}$$


In [5]:
import torch
import torch.nn as nn


#####################################
# Chapter 3
#####################################

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x):
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift


class RMSNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.emb_dim = emb_dim
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()

    def forward(self, x):
        means = x.pow(2).mean(dim=-1, keepdim=True)
        x_normed = x * torch.rsqrt(means + self.eps)
        return (x_normed * self.weight).to(dtype=x.dtype)

##  GELU функциясын SiLU активациясына ауыстыру
##  Заменить функцию активации GELU на SiLU

* Llama GELU орнына SiLU (Swish) активация функциясын пайдаланады
$$
\text{silu}(x) = x \cdot \sigma(x), \quad \text{мұндағы} \quad \sigma(x) \text{ — логистикалық сигмоида.}
$$
---
* В модели Llama используется функция активации SiLU (вместо GELU), также известная как функция Swish:

$$
\text{silu}(x) = x \cdot \sigma(x), \quad \text{где} \quad \sigma(x) \text{ — логистическая сигмоида.}
$$


In [6]:
#####################################
# Chapter 3
#####################################

# class GELU(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         return 0.5 * x * (1 + torch.tanh(
#             torch.sqrt(torch.tensor(2.0 / torch.pi)) *
#             (x + 0.044715 * torch.pow(x, 3))
#         ))


class SiLU(nn.Module):
    def __init__(self):
        super(SiLU, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)

##  FeedForward модулін жаңарту
##  Обновление модуля FeedForward


* Шын мәнінде, Llama SiLU функциясының “Gated Linear Unit” (GLU) нұсқасы болып табылатын SwiGLU-ды қолданады, бұл `FeedForward` модулінің құрылымын сәл өзгеше етеді.
* SwiGLU feedforward қабатында **gating** (қақпалау) механизмін қолданады, оның формуласы:

$$\text{SwiGLU}(x) = \text{SiLU}(\text{Linear}_1(x)) * (\text{Linear}_2(x))$$

* Мұнда $\text{Linear}_1$ және $\text{Linear}_2$ — екі сызықтық қабат, ал $*$ — элемент бойынша көбейту (element-wise multiplication).
* Үшінші сызықтық қабат — $\text{Linear}_3$ — осы gated активациядан кейін қолданылады.
---
* На самом деле, Llama использует вариант функции SiLU под названием **SwiGLU**, которая является модификацией “Gated Linear Unit” (GLU). Это приводит к немного другой структуре модуля `FeedForward`.
* SwiGLU применяет **механизм «врат» (gating)** в полносвязном слое, формула выглядит так:

$$\text{SwiGLU}(x) = \text{SiLU}(\text{Linear}_1(x)) * (\text{Linear}_2(x))$$

* Здесь $\text{Linear}_1$ и $\text{Linear}_2$ — это два линейных слоя, а $*$ обозначает покомпонентное (element-wise) умножение.
* Третий линейный слой, $\text{Linear}_3$, применяется после этой gated-активации.


In [7]:
#####################################
# Chapter 3
#####################################
# class FeedForward(nn.Module):
#     def __init__(self, cfg):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
#             GELU(),
#             nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
#         )

#     def forward(self, x):
#         return self.layers(x)

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()

    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        x = self.silu(x_fc1) * x_fc2
        return self.fc3(x)

* Айта кету керек, біз жоғарыда `dtype=cfg["dtype"]` параметрін де қостық. Жадты үнемдеу үшін бұл модельді кейінірек төмен дәлдіктегі форматтарда (lower precision) тікелей жүктеуге мүмкіндік береді (модельді бастапқы 32-бит форматында жасап, кейін түрлендірудің орнына).
* Сондай-ақ, біз `bias=False` деп орнаттық, себебі Llama модельдері **bias** бірліктерін қолданбайды.
---
* Обратите внимание, что мы также добавили параметр `dtype=cfg["dtype"]`, который позволит нам загружать модель напрямую в форматах с пониженной точностью, чтобы уменьшить использование памяти (вместо создания модели в исходном 32-битном формате и последующего преобразования).
* Мы также установили `bias=False`, поскольку модель Llama не использует **bias**-элементы.


* GPT моделінде позициялық эмбеддингтер келесідей түрде жүзеге асырылады:

```python
self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
```

* Дәстүрлі абсолюттік позициялық эмбеддингтерге қарағанда, **Llama** rotary position embeddings (RoPE) әдісін қолданады, бұл модельге **абсолюттік және салыстырмалы позициялық ақпаратты бір уақытта** үйренуге мүмкіндік береді.
* RoPE туралы негізгі мақала: [RoFormer: Enhanced Transformer with Rotary Position Embedding (2021)](https://arxiv.org/abs/2104.09864).
* RoPE екі эквивалентті тәсілмен жүзеге асырылуы мүмкін: *split-halves* нұсқасы және *interleaved even/odd* нұсқасы арқылы;
* Бұл кодта RoPE-тің *split-halves* тәсілі қолданылады.
---
* В модели GPT позиционные эмбеддинги реализованы следующим образом:

```python
self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
```

* В отличие от традиционных абсолютных позиционных эмбеддингов, **Llama** использует **rotary position embeddings (RoPE)**, которые позволяют одновременно учитывать **абсолютную и относительную позиционную информацию**.
* Основная статья по RoPE: [RoFormer: Enhanced Transformer with Rotary Position Embedding (2021)](https://arxiv.org/abs/2104.09864).
* RoPE можно реализовать двумя эквивалентными способами: *split-halves* и *interleaved even/odd*;
* В этом коде используется метод *split-halves*.

In [8]:
def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096):
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # Compute the inverse frequencies
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # Generate position indices
    positions = torch.arange(context_length)

    # Compute the angles
    angles = positions.unsqueeze(1) * inv_freq.unsqueeze(0)  # Shape: (context_length, head_dim // 2)

    # Expand angles to match the head_dim
    angles = torch.cat([angles, angles], dim=1)  # Shape: (context_length, head_dim)

    # Precompute sine and cosine
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin

def compute_rope(x, cos, sin):
    # x: (batch_size, num_heads, seq_len, head_dim)
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 == 0, "Head dimension must be even"

    # Split x into first half and second half
    x1 = x[..., : head_dim // 2]  # First half
    x2 = x[..., head_dim // 2 :]  # Second half

    # Adjust sin and cos shapes
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, seq_len, head_dim)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # Apply the rotary transformation
    rotated = torch.cat((-x2, x1), dim=-1)
    x_rotated = (x * cos) + (rotated * sin)

    return x_rotated.to(dtype=x.dtype)

* Төменде `q` және `k` тензорларына RoPE қолданудың мысалы келтірілген:
---
* Ниже приведён пример применения RoPE к тензорам `q` и `k`:

In [9]:
# Settings
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

# Instantiate RoPE parameters
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# Dummy query and key tensors
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.randn(batch_size, num_heads, context_len, head_dim)

# Apply rotary position embeddings
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)

## RoPE-ті MultiHeadAttention модуліне қосу
## Добавление RoPE в модуль MultiHeadAttention

* Маңыздысы — GPT позициялық эмбеддингтерді **кіріс векторларына** қолданады, ал Llama позициялық ақпаратты **self-attention механизміндегі query және key векторларына айналдыру (rotation)** арқылы енгізеді.
* Мұнда біз `MultiHeadAttention` класын тиісті RoPE кодымен өзгертеміз.
* Сонымен қатар, `qkv_bias` параметрін алып тастап, `bias=False` мәнін  орнатамыз.
* Сондай-ақ, кейін модельді төмен дәлдіктегі форматта (lower precision) инициализациялау үшін `dtype` параметрін қосамыз.
---
* Важно отметить, что GPT применяет позиционные эмбеддинги **к входным векторам**, тогда как Llama добавляет позиционную информацию, **вращая (rotating)** векторы query и key прямо внутри механизма самовнимания (self-attention).
* Здесь мы модифицируем класс `MultiHeadAttention`, добавляя соответствующий код для RoPE.
* Кроме того, мы удаляем параметр `qkv_bias` и  задаём значение `bias=False`.
* Также мы добавляем параметр `dtype`, чтобы в дальнейшем можно было инициализировать модель в формате с пониженной точностью (lower precision).


In [10]:
#####################################
# Chapter 2
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):  # ,dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        ################################### NEW ###################################
        # Set bias=False and dtype=dtype for all linear layers below
        ###########################################################################
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)  # Linear layer to combine head outputs
        # self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
        ###########################################################################


    def forward(self, x):

        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        ################################### NEW ###################################
        keys = compute_rope(keys, self.cos, self.sin)
        queries = compute_rope(queries, self.cos, self.sin)
        ###########################################################################

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

* Төменде `MultiHeadAttention` модулін мысалын  қолдану үлгісі көрсетілген:
---
* Ниже приведён пример использования модуля `MultiHeadAttention` на примере входных данных:

In [11]:
# Settings
batch_size = 1
context_len = 100
max_context_len = 4096
embed_dim = 128
num_heads = 4


example_batch = torch.randn((batch_size, context_len, embed_dim))

mha = MultiHeadAttention(
    d_in=embed_dim,
    d_out=embed_dim,
    context_length=max_context_len,
    num_heads=num_heads
)

mha(example_batch)

del mha  # delete to free up memory

## TransformerBlock модулін жаңарту
## Обновление модуля TransformerBlock

Енді біз жоғарыда жүзеге асырған кодты пайдалану үшін `TransformerBlock` модулін жаңарта аламыз:
  * **LayerNorm** орнына **RMSNorm** қолданамыз
  * **Dropout** қабатын алып тастаймыз
  * `qkv_bias` параметрін жоямыз
  * `dtype` параметрін қосамыз
---
Теперь мы можем обновить модуль `TransformerBlock`, чтобы использовать код, реализованный выше:
  * заменяем **LayerNorm** на **RMSNorm**
  * удаляем **Dropout**
  * убираем параметр `qkv_bias`
  * добавляем параметр `dtype`


In [12]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dtype=cfg["dtype"]  # NEW
            # dropout=cfg["drop_rate"],
            # qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)

        ################################### NEW ###################################
        # self.norm1 = LayerNorm(cfg["emb_dim"])
        # self.norm2 = LayerNorm(cfg["emb_dim"])
        self.norm1 = RMSNorm(cfg["emb_dim"])
        self.norm2 = RMSNorm(cfg["emb_dim"])
        ###########################################################################

        # self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

##  Модель класын жаңарту
##  Обновление класса модели


* Біздің Llama моделіміз дерлік дайын, тек `TransformerBlock` айналасындағы модель кодын жаңарту қажет:
  * Енді RoPE эмбеддингтерін қолданғандықтан, **абсолюттік позициялық эмбеддингтерді** алып тастаймыз
  * **LayerNorm** орнына **RMSNorm** қолданамыз
  * **Dropout** қабатын алып тастаймыз
  * **dtype** параметрін қосамыз
---
* Наша модель Llama почти завершена — осталось лишь обновить код модели вокруг `TransformerBlock`:
  * удаляем **абсолютные позиционные эмбеддинги**, так как теперь используем RoPE
  * заменяем **LayerNorm** на **RMSNorm**
  * удаляем **Dropout**
  * добавляем параметр **dtype**


In [13]:
# class GPTModel(nn.Module):
class Llama2Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
        # self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        ################################### NEW ###################################
        # self.final_norm = LayerNorm(cfg["emb_dim"])
        self.final_norm = RMSNorm(cfg["emb_dim"])
        ###########################################################################
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

    def forward(self, in_idx):
        # batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds  # + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

##  Модельді инициализациялау
##  Инициализация модели


In [14]:
LLAMA2_CONFIG_7B = {
    "vocab_size": 32000,     # Vocabulary size
    "context_length": 4096,  # Context length
    "emb_dim": 4096,         # Embedding dimension
    "n_heads": 32,           # Number of attention heads
    "n_layers": 32,          # Number of layers
    "hidden_dim": 11008,     # NEW: Size of the intermediate dimension in FeedForward
    "dtype": torch.bfloat16  # NEW: Lower-precision dtype to reduce memory usage
}

In [15]:
model = Llama2Model(LLAMA2_CONFIG_7B)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 6,738,415,616


* Жоғарыда көрсетілгендей, модельде **6.7 миллиард параметр** бар (әдетте оны жуықтап **7B модель** деп атайды).
* Сонымен қатар, төмендегі код арқылы осы модельдің жады көлеміне қойылатын талаптарын есептеуге болады:
---
* Как показано выше, модель содержит **6,7 миллиарда параметров** (обычно округляется и называется **моделью 7B**).
* Кроме того, с помощью приведённого ниже кода можно вычислить требования к памяти для этой модели:


In [16]:
def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    for param in model.parameters():
        # Calculate total number of elements per parameter
        param_size = param.numel()
        total_params += param_size
        # Check if gradients are stored for this parameter
        if param.requires_grad:
            total_grads += param_size

    # Calculate buffer size (non-parameters that require memory)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # Size in bytes = (Number of elements) * (Size of each element in bytes)
    # We assume parameters and gradients are stored in the same type as input dtype
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # Convert bytes to gigabytes
    total_memory_gb = total_memory_bytes / (1024**3)

    return total_memory_gb

print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

float32 (PyTorch default): 52.33 GB
bfloat16: 26.17 GB


# Llama 2 архитектурасын Llama 3.2 архитектурасына түрлендіру
# Преобразование архитектуры Llama 2  в Llama 3.2

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/gpt2-to-llama2-llama3.webp?1">

In [17]:
from importlib.metadata import version

pkgs = [
    "tiktoken",         # to implement the tokenizer
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

tiktoken version: 0.8.0
torch version: 2.5.1


##  RoPE жаңарту
##  Модифицикация RoPE

* **Llama 3** моделі **Llama 2** сияқты **rotary position embeddings (RoPE)** әдісін қолданады (толық түсіндірме үшін [RoPE мақаласын](https://arxiv.org/abs/2104.09864) қараңыз).
* Алайда RoPE параметрлерінде аздаған айырмашылықтар бар:

  * **Llama 3** енді **8,192 токенге дейін** қолдайды — бұл **Llama 2**-дегі (4,096) саннан екі есе көп.
  * Төмендегі формуладағы **RoPE $\theta$** айнымалысының базалық мәні **10,000**-нан (**Llama 2**) **500,000**-ға (**Llama 3**) арттырылған ([RoPE мақаласынан](https://arxiv.org/abs/2104.09864) бейімделген формула):

$$\Theta = \left\{\theta_i = \text{base}^{\frac{-2(i-1)}{d}}, i \in \left[1, 2, ..., d/2\right]\right\}$$

* Бұл $\theta$ мәндері — **айналдыру матрицасындағы бұрыштарды анықтайтын алдын ала берілген параметрлер**, мұнда $d$ — эмбеддинг кеңістігінің өлшемі.
* **Базаны 10,000-нан 500,000-ға арттыру** жиіліктердің (немесе айналу бұрыштарының) өлшемдерін  баяу өзгеруіне әкеледі, яғни жоғары өлшемдер бұрынғыға қарағанда **үлкен бұрыштармен** байланысады (бұл — жиіліктердің "декомпрессиясы").
---
* **Llama 3** использует **rotary position embeddings (RoPE)**, аналогично **Llama 2** (подробное объяснение можно найти в [статье о RoPE](https://arxiv.org/abs/2104.09864)).
* Однако в настройках RoPE есть несколько отличий:

  * **Llama 3** теперь поддерживает до **8 192 токенов**, что вдвое больше, чем в **Llama 2** (4 096).
  * Базовое значение параметра **RoPE $\theta$** увеличено с **10 000** (в Llama 2) до **500 000** (в Llama 3) в следующем уравнении (адаптированном из [статьи RoPE](https://arxiv.org/abs/2104.09864)):

$$\Theta = \left\{\theta_i = \text{base}^{\frac{-2(i-1)}{d}}, i \in \left[1, 2, ..., d/2\right]\right\}$$

* Эти значения $\theta$ — это набор заранее определённых параметров, которые определяют углы вращения в роторной матрице, где $d$ — размерность пространства эмбеддингов.
* **Увеличение базы с 10 000 до 500 000** приводит к тому, что частоты (или углы вращения) уменьшаются медленнее по измерениям, то есть **высшие измерения** будут связаны с **большими углами**, чем раньше (по сути, это «декомпрессия» частот).


In [18]:
import torch

def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096, freq_config=None):
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # Compute the inverse frequencies
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    ################################ NEW ###############################################
    # Frequency adjustments
    if freq_config is not None:
        low_freq_wavelen = freq_config["original_context_length"] / freq_config["low_freq_factor"]
        high_freq_wavelen = freq_config["original_context_length"] / freq_config["high_freq_factor"]

        wavelen = 2 * torch.pi / inv_freq

        inv_freq_llama = torch.where(
            wavelen > low_freq_wavelen, inv_freq / freq_config["factor"], inv_freq
        )

        smooth_factor = (freq_config["original_context_length"] / wavelen - freq_config["low_freq_factor"]) / (
            freq_config["high_freq_factor"] - freq_config["low_freq_factor"]
        )

        smoothed_inv_freq = (
            (1 - smooth_factor) * (inv_freq / freq_config["factor"]) + smooth_factor * inv_freq
        )

        is_medium_freq = (wavelen <= low_freq_wavelen) & (wavelen >= high_freq_wavelen)
        inv_freq_llama = torch.where(is_medium_freq, smoothed_inv_freq, inv_freq_llama)
        inv_freq = inv_freq_llama
    ####################################################################################


    # Generate position indices
    positions = torch.arange(context_length)

    # Compute the angles
    angles = positions.unsqueeze(1) * inv_freq.unsqueeze(0)  # Shape: (context_length, head_dim // 2)

    # Expand angles to match the head_dim
    angles = torch.cat([angles, angles], dim=1)  # Shape: (context_length, head_dim)

    # Precompute sine and cosine
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin

In [19]:
# Instantiate RoPE parameters

llama_2_context_len = 4096
llama_3_context_len = 8192

llama_2_theta_base = 10_000
llama_3_theta_base = 500_000

# Settings
batch_size = 2
num_heads = 4
head_dim = 16

# Instantiate RoPE parameters
cos, sin = precompute_rope_params(
    head_dim=head_dim,
    theta_base=llama_3_theta_base,
    context_length=llama_3_context_len
)

# Dummy query and key tensors
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, llama_3_context_len, head_dim)
keys = torch.randn(batch_size, num_heads, llama_3_context_len, head_dim)

# Apply rotary position embeddings
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)

## Топталған сұрау арқылы назар аудару (Grouped-query attention)
## Группированное внимание по запросам (Grouped-query attention)

* Бұл бөлімде біз **multi-head attention (MHA)** механизмін **grouped-query attention (GQA)** деп аталатын балама тәсілмен ауыстырамыз.
* Қысқаша айтқанда, **GQA** — бұл **есептеу және параметр тұрғысынан тиімдірек** MHA нұсқасы.
* **GQA**-да key және value проекцияларының санын азайту үшін оларды бірнеше attention head арасында ортақ пайдаланамыз.
* Әрбір attention head өзіне тән **query** векторына ие, бірақ олардың барлығы **бір топтағы key және value** векторларына назар аударады.
* Төменде 2 key-value тобы (kv-groups) бар **GQA** мысалы көрсетілген:
---
* В этом разделе мы заменяем механизм **multi-head attention (MHA)** на альтернативный — **grouped-query attention (GQA)**.
* Вкратце, **GQA** можно рассматривать как **более вычислительно и параметрически эффективную** версию MHA.
* В **GQA** уменьшается количество проекций key и value за счёт их **совместного использования несколькими головами внимания**.
* Каждая голова внимания по-прежнему имеет свой собственный **query**, но все они обращаются к **одной группе ключей и значений**.
* Ниже представлена иллюстрация **GQA** с двумя группами key-value (kv-groups):


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/grouped-query-attention.webp" width="500px">

* GQA-ның негізгі идеясы — кілт-мағына (key-value) жұптарына қатысатын бірегей сұраныс топтарының (query groups) санын азайту. Бұл матрица көбейтулерінің кейбірінің өлшемін және MHA-дегі параметрлер санын азайтып, модель өнімділігін айтарлықтай төмендетпей тиімділікті арттырады.
* GQA коды MHA-ға өте ұқсас (төменде "NEW" деп белгіленген бөлімдер арқылы өзгертулер көрсетілген).
* Қысқаша айтқанда, GQA-дағы негізгі өзгеріс — әрбір сұраныс тобы оған сәйкес келетін бас (head) санына сәйкес келуі үшін қайталануы керек, бұл төменде көрсетілгендей жүзеге асырылады.
---
* Основная идея GQA заключается в сокращении количества уникальных групп запросов (query groups), обращающихся к парам ключ-значение (key-value), что уменьшает размер некоторых матричных умножений и количество параметров в MHA, не снижая при этом существенно качество модели.
* Код GQA очень похож на MHA (изменения выделены ниже в разделах "NEW").
* Вкратце, главное изменение в GQA состоит в том, что каждая группа запросов должна быть повторена столько раз, сколько голов (heads) с ней связано, как показано в реализации ниже.


In [20]:
import torch.nn as nn


class GroupedQueryAttention(nn.Module):
    def __init__(
            self, d_in, d_out, num_heads,
            num_kv_groups,       # NEW
            dtype=None
        ):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        assert num_heads % num_kv_groups == 0, "num_heads must be divisible by num_kv_groups"  # NEW

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        ############################# NEW  #############################
        # self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        # self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, num_kv_groups * self.head_dim, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, num_kv_groups * self.head_dim, bias=False, dtype=dtype)
        self.num_kv_groups = num_kv_groups
        self.group_size = num_heads // num_kv_groups
        ################################################################

        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)


    def forward(self, x, mask=None, cos=None, sin=None):
        ##################### NEW  #####################
        # The forward method now accepts `mask` instead of accessing it via self.mask.
        # Also, we now have cos and sin as input for RoPE
        ################################################    
        b, num_tokens, d_in = x.shape

        queries = self.W_query(x)  # Shape: (b, num_tokens, d_out)
        keys = self.W_key(x)  # Shape: (b, num_tokens, num_kv_groups * head_dim)
        values = self.W_value(x)  # Shape: (b, num_tokens, num_kv_groups * head_dim)

        # Reshape queries, keys, and values
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        ##################### NEW  #####################
        # keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        # values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        keys = keys.view(b, num_tokens, self.num_kv_groups, self.head_dim)
        values = values.view(b, num_tokens, self.num_kv_groups, self.head_dim)
        ################################################

        # Transpose keys, values, and queries
        keys = keys.transpose(1, 2)  # Shape: (b, num_kv_groups, num_tokens, head_dim)
        values = values.transpose(1, 2)  # Shape: (b, num_kv_groups, num_tokens, head_dim)
        queries = queries.transpose(1, 2)  # Shape: (b, num_heads, num_tokens, head_dim)

        ##################### NEW #####################
        # Apply RoPE
        if cos is not None:
            keys = compute_rope(keys, cos, sin)
            queries = compute_rope(queries, cos, sin)
        ################################################

        ##################### NEW  #####################
        # Expand keys and values to match the number of heads
        # Shape: (b, num_heads, num_tokens, head_dim)

        keys = keys.repeat_interleave(self.group_size, dim=1)  # Shape: (b, num_heads, num_tokens, head_dim)
        values = values.repeat_interleave(self.group_size, dim=1)  # Shape: (b, num_heads, num_tokens, head_dim)
        # For example, before repeat_interleave along dim=1 (query groups):
        #   [K1, K2]
        # After repeat_interleave (each query group is repeated group_size times):
        #   [K1, K1, K2, K2]
        # If we used regular repeat instead of repeat_interleave, we'd get:
        #   [K1, K2, K1, K2]
        ################################################

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        # Shape: (b, num_heads, num_tokens, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        ##################### NEW #####################
        # Create mask on the fly
        if mask is None:
            mask = torch.triu(torch.ones(num_tokens, num_tokens, device=x.device, dtype=torch.bool), diagonal=1)
        ################################################
    
        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        assert keys.shape[-1] == self.head_dim

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

**LLama 2**

In [21]:
# Settings
batch_size = 1
context_len = 3000
max_context_len = 8192
embed_dim = 4096
num_heads = 32


example_batch = torch.randn((batch_size, context_len, embed_dim))

mha = MultiHeadAttention(
    d_in=embed_dim,
    d_out=embed_dim,
    context_length=max_context_len,
    num_heads=num_heads
)

mha(example_batch)

print("W_key:", mha.W_key.weight.shape)
print("W_value:", mha.W_value.weight.shape)
print("W_query:", mha.W_query.weight.shape)

W_key: torch.Size([4096, 4096])
W_value: torch.Size([4096, 4096])
W_query: torch.Size([4096, 4096])


* Енді, егер біз grouped-query attention механизмын 8 kv-топқа бөлсек (Llama 3 8B дәл осылай жасайды), онда кілт (key) және мән (value) матрицаларындағы жолдар санының 4 есеге азайғанын байқаймыз (өйткені 32 attention басы 8 kv-топқа бөлінгенде 4 шығады).
---
* Теперь, если мы используем grouped-query attention с 8 kv-группами (именно столько используется в Llama 3 8B), можно увидеть, что количество строк в матрицах ключей (key) и значений (value) уменьшается в 4 раза (так как 32 головы внимания делятся на 8 kv-групп, получаем 4).


**Llama 3**

In [22]:
gqa = GroupedQueryAttention(
    d_in=embed_dim,
    d_out=embed_dim,
    num_heads=num_heads,
    num_kv_groups=8,
)

gqa(example_batch)

print("W_key:", gqa.W_key.weight.shape)
print("W_value:", gqa.W_value.weight.shape)
print("W_query:", gqa.W_query.weight.shape)

W_key: torch.Size([1024, 4096])
W_value: torch.Size([1024, 4096])
W_query: torch.Size([4096, 4096])


* Қосымша ескерту ретінде: GroupedQueryAttention-ды стандартты көпбасты attention (multi-head attention) механизміне тең ету үшін, сұраныс топтарының санын (`num_kv_groups`) бастар санына (`num_heads`) тең етіп орнатуға болады.
* Соңында, төменде параметрлер санын салыстырып көрейік:
---
* В качестве примечания: чтобы сделать GroupedQueryAttention эквивалентным стандартному механизму multi-head attention, можно установить количество групп запросов (`num_kv_groups`) равным количеству голов (`num_heads`).
* Наконец, давайте сравним количество параметров ниже:


In [23]:
print("Total number of parameters:")

mha_total_params = sum(p.numel() for p in mha.parameters())
print(f"MHA: {mha_total_params:,}")

gqa_total_params = sum(p.numel() for p in gqa.parameters())
print(f"GQA: {gqa_total_params:,}")

Total number of parameters:
MHA: 67,108,864
GQA: 41,943,040


In [24]:
# Free up memory:
del mha
del gqa

## LLama 3 моделын жасау
***
## Создание Llama 3 модели


In [25]:
# class Llama2Model(nn.Module):
class Llama3Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = RMSNorm(cfg["emb_dim"], eps=1e-5)
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

        #################### NEW #####################
        cos, sin = precompute_rope_params(
            head_dim=cfg["emb_dim"] // cfg["n_heads"],
            theta_base=cfg["rope_base"],
            context_length=cfg["context_length"],
            freq_config=cfg["rope_freq"]
        )
        
        self.register_buffer("cos", cos, persistent=False)
        self.register_buffer("sin", sin, persistent=False)
        ##############################################

        self.cfg = cfg

    def forward(self, in_idx):
        tok_embeds = self.tok_emb(in_idx)
        x = tok_embeds

        #################### NEW #####################
        num_tokens = x.shape[1]
        mask = torch.triu(torch.ones(num_tokens, num_tokens, device=x.device, dtype=torch.bool), diagonal=1)
        ##############################################
        
        for block in self.trf_blocks:
            x = block(x, mask, self.cos, self.sin)
        x = self.final_norm(x)
        logits = self.out_head(x.to(self.cfg["dtype"]))
        return logits

In [26]:
LLAMA3_CONFIG_8B = {
    "vocab_size": 128_256,   # NEW: Larger vocabulary size
    "context_length": 8192,  # NEW: Larger context length
    "emb_dim": 4096,         # Embedding dimension
    "n_heads": 32,           # Number of attention heads
    "n_layers": 32,          # Number of layers
    "hidden_dim": 14_336,    # NEW: Larger size of the intermediate dimension in FeedForward
    "n_kv_groups": 8,        # NEW: Key-Value groups for grouped-query attention
    "rope_base": 500_000.0,  # NEW: The base in RoPE's "theta" was increased to 500_000
    "rope_freq": None,       # NEW: Additional configuration for adjusting the RoPE frequencies
    "dtype": torch.bfloat16  # Lower-precision dtype to reduce memory usage
}

In [27]:
model = Llama3Model(LLAMA3_CONFIG_8B)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 8,835,567,616


In [28]:
def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    for param in model.parameters():
        # Calculate total number of elements per parameter
        param_size = param.numel()
        total_params += param_size
        # Check if gradients are stored for this parameter
        if param.requires_grad:
            total_grads += param_size

    # Calculate buffer size (non-parameters that require memory)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # Size in bytes = (Number of elements) * (Size of each element in bytes)
    # We assume parameters and gradients are stored in the same type as input dtype
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # Convert bytes to gigabytes
    total_memory_gb = total_memory_bytes / (1024**3)

    return total_memory_gb

print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

float32 (PyTorch default): 74.09 GB
bfloat16: 37.04 GB


# Llama 3.1 8B

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/llama3-to-llama31.webp" width="700px">

* Архитектура бірдей, тек төмендегі конфигурация файлында көрсетілгендей RoPE жиіліктерін қайта масштабтау ғана өзгеше.
---
* Архитектура идентична, единственное изменение — это перемасштабирование частот RoPE, как указано в конфигурационном файле ниже.


In [29]:
LLAMA31_CONFIG_8B = {
    "vocab_size": 128_256,      # Vocabulary size
    "context_length": 131_072,  # NEW: Larger supported context length
    "emb_dim": 4096,            # Embedding dimension
    "n_heads": 32,              # Number of attention heads
    "n_layers": 32,             # Number of layers
    "hidden_dim": 14_336,       # Size of the intermediate dimension in FeedForward
    "n_kv_groups": 8,           # Key-Value groups for grouped-query attention
    "rope_base": 500_000.0,     # The base in RoPE's "theta"
    "dtype": torch.bfloat16,    # Lower-precision dtype to reduce memory usage
    "rope_freq": {              # NEW: RoPE frequency scaling
        "factor": 8.0,
        "low_freq_factor": 1.0,
        "high_freq_factor": 4.0,
        "original_context_length": 8192,
    }
}

* Бұрын кодта көргеніміздей, RoPE әдісі позициялық ақпаратты тікелей attention механизміне енгізу үшін синусоидалы функцияларды (синус және косинус) пайдаланады.
* Llama 3.1-де қосымша конфигурация арқылы кері жиілік (inverse frequency) есептеулеріне қосымша түзетулер енгізіледі.
* Бұл түзетулер әртүрлі жиілік компоненттерінің позициялық ендірулерге (positional embeddings) қалай әсер ететінін өзгертеді (бұл тақырыпты кейінірек толығырақ талқылаймыз).
---
* Как мы уже видели в коде ранее, метод RoPE использует синусоидальные функции (синус и косинус), чтобы встроить позиционную информацию напрямую в механизм внимания.
* В Llama 3.1, благодаря дополнительной конфигурации, вводятся дополнительные корректировки в расчёт обратных частот (inverse frequency).
* Эти корректировки влияют на то, как разные частотные компоненты вносят вклад в позиционные эмбеддинги (подробное объяснение этого оставим на потом).

In [30]:
import gc
# free up memory
del model

gc.collect()  # Run Python garbage collector

if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [31]:
from pathlib import Path
import os
import tiktoken
from tiktoken.load import load_tiktoken_bpe


class Tokenizer:
    """Thin wrapper around tiktoken that keeps track of Llama-3 special IDs."""
    def __init__(self, model_path):
        if not os.path.isfile(model_path):
            raise FileNotFoundError(model_path)

        mergeable = load_tiktoken_bpe(model_path)

        # hard-coded from Meta's tokenizer.json
        self.special = {
            "<|begin_of_text|>": 128000,
            "<|end_of_text|>": 128001,
            "<|start_header_id|>": 128006,
            "<|end_header_id|>": 128007,
            "<|eot_id|>": 128009,
        }
        self.special.update({f"<|reserved_{i}|>": 128002 + i
                             for i in range(256)
                             if 128002 + i not in self.special.values()})

        self.model = tiktoken.Encoding(
            name=Path(model_path).name,
            pat_str=r"(?i:'s|'t|'re|'ve|'m|'ll|'d)"
                    r"|[^\r\n\p{L}\p{N}]?\p{L}+"
                    r"|\p{N}{1,3}"
                    r"| ?[^\s\p{L}\p{N}]+[\r\n]*"
                    r"|\s*[\r\n]+"
                    r"|\s+(?!\S)"
                    r"|\s+",
            mergeable_ranks=mergeable,
            special_tokens=self.special,
        )

    def encode(self, text, bos=False, eos=False):
        ids = ([self.special["<|begin_of_text|>"]] if bos else []) \
              + self.model.encode(text)
        if eos:
            ids.append(self.special["<|end_of_text|>"])
        return ids

    def decode(self, ids):
        return self.model.decode(ids)

In [31]:
from huggingface_hub import login
import os

login(token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [34]:
from huggingface_hub import hf_hub_download
tokenizer_file_path = hf_hub_download(
    repo_id="meta-llama/Llama-3.1-8B",
    filename="original/tokenizer.model",
    local_dir="Llama-3.1-8B"
)

tokenizer = Tokenizer(tokenizer_file_path)

original/tokenizer.model:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

In [ ]:
model = Llama3Model(LLAMA31_CONFIG_8B)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

# Llama 3.2 1B


* Llama 3.2  моделінің коды Llama 3.1-ге ұқсас, тек модельдің өлшемі кішірейген (1B және 3B нұсқалары бар).
* Тағы бір тиімділігі — салмақтарды байланыстыруды (weight tying) қайта қосу болды (бұл идея алғаш рет GPT-2 архитектурасында қолданылған); мұнда кіріс (token embedding) қабаты мен шығыс қабатында бірдей салмақ параметрлері қайта пайдаланылады.
* Llama 3.2 1B моделі өте ықшам, себебі ол тіпті көптеген мобильді құрылғыларда да жұмыс істей алады.
* Llama 3.1 8B және Llama 3.2 1B модельдерінің архитектуралық айырмашылықтары төмендегі суретте көрсетілген.
---
* Код  модели Llama 3.2 похож на Llama 3.1, за исключением того, что размер модели уменьшился (существуют версии на 1B и 3B параметров).
* Ещё одно улучшение эффективности заключается в возвращении приёма «weight tying» (связывания весов) — концепции, впервые использованной в архитектуре GPT-2; здесь те же значения весовых параметров используются как в входном (token embedding) слое, так и в выходном.
* Малый размер модели Llama 3.2 1B делает её весьма удобной, поскольку она может работать даже на многих мобильных устройствах.
* Архитектурные различия между Llama 3.1 8B и Llama 3.2 1B показаны на рисунке ниже.


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/llama31-to-llama32.webp?1" width="700px">

* Жоғарыдағы суретке қарағанда, Llama 3.1 8B мен Llama 3.2 1B архитектураларының негізгі айырмашылығы — олардың өлшемдерінде.
* Қосымша шағын өзгеріс — RoPE қайта масштабтау коэффициентінің ұлғаюы, бұл төмендегі конфигурация файлында көрсетілген.
---
* Как видно из приведённого выше рисунка, основное различие между архитектурами Llama 3.1 8B и Llama 3.2 1B заключается в их размерах.
* Небольшое дополнительное изменение — увеличение коэффициента рескейлинга (перемасштабирования) RoPE, что отражено в конфигурационном файле ниже.


* Төменде біз Llama 3.1 8B бөліміндегі кодты қайта қолданып, Llama 3.2 1B моделін жүктей аламыз.
* Тағы да ескерту ретінде: Llama 3.2 моделі Llama 3.1 отбасынан бөлек болғандықтан, [meta-llama/Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B) репозиторийіне кіріп, лицензиялық шарттармен келісу қажет — әйтпесе Hugging Face access token арқылы жүктеу жұмыс істемейді.
* Кеңес: Қарапайымдылық үшін төменде тек базалық модель жүктеледі, бірақ `"meta-llama/Llama-3.2-1B"` атауын `"meta-llama/Llama-3.2-1B-Instruct"` деп ауыстыру арқылы нұсқаулықпен (instruction) бапталған нұсқасын да қолдануға болады.
---
* Ниже мы можем повторно использовать код из раздела Llama 3.1 8B, чтобы загрузить модель Llama 3.2 1B.
* Опять же, поскольку семейство Llama 3.2 отличается от Llama 3.1, необходимо перейти в репозиторий [meta-llama/Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B) и принять условия лицензии, чтобы ваш токен доступа Hugging Face работал при загрузке.
* Совет: Для простоты ниже мы загружаем только базовую модель, но вы также можете использовать инструкционно дообученную версию, заменив `"meta-llama/Llama-3.2-1B"` на `"meta-llama/Llama-3.2-1B-Instruct"`.


# Testing models

In [ ]:
import torch
from google.colab import userdata
from transformers import pipeline
from huggingface_hub import login

login(token=userdata.get('HF_TOKEN'))

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="mps",
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)

In [ ]:
print(outputs[0]["generated_text"][-1]['content'])